In [1]:
import sys
sys.path.append('/scratch/programming/pipeline/preprocessing-pipeline/in_development/Will')
sys.path.append('/scratch/programming/pipeline/pipeline_utility/src')
from cell_extractor.CellDetectorTrainer import CellDetectorTrainer
from cell_extractor.DetectorMetrics import AnnotationProximityTool,DetectorMetricsDK55
import pandas as pd

1.5.0


SyntaxError: invalid syntax (DetectorMetrics.py, line 117)

In [2]:
trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=2300)
trainer.list_detectors()

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


['detector_round_2_threshold_2300.pkl',
 'detector_round_2_threshold_2200.pkl',
 'detector_round_2_threshold_2100.pkl',
 'detector_round_2.pkl',
 'detector_round_2_threshold_2000.pkl',
 'detector_round_1.pkl',
 'detector_round_2_threshold_2700.pkl']

In [3]:
all_detections = []
for threshold in [2000,2100,2200,2300,2700]:
    trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=threshold)
    detections = trainer.load_detections()
    detections = detections[['row','col','section','predictions']]
    detections['name'] = str(threshold)
    all_detections.append(detections)
all_detections = pd.concat(all_detections)

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


In [40]:
sures = all_detections[all_detections.predictions==2]
unsures = all_detections[all_detections.predictions==0]
sures = sures.drop('predictions',axis=1)
unsures = unsures.drop('predictions',axis=1)
sures.columns = ['x','y','section','name']
unsures.columns = ['x','y','section','name']

In [6]:
util = AnnotationProximityTool()
util.set_annotations_to_compare(sures)
util.find_equivalent_points()
pair_id = [pair[0] for pair in util.pairs.values()]
sure_cells = util.annotations_to_compare.iloc[pair_id]

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 62614
after removing duplicates 12740


In [43]:
util = AnnotationProximityTool()
util.set_annotations_to_compare(unsures)
util.find_equivalent_points()
pair_id = [pair[0] for pair in util.pairs.values()]
unsure_cells = util.annotations_to_compare.iloc[pair_id]

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 1486
after removing duplicates 553


In [47]:
sure_cells = sure_cells.drop('name',axis=1)
unsure_cells = unsure_cells.drop('name',axis=1)
sure_cells['prediction'] = 2
unsure_cells['prediction'] = 0

In [48]:
result = pd.concat([sure_cells,unsure_cells])
result.head()

,x,y,section,prediction
0,18851,37633,132,2
1,17423,38133,145,2
2,17638,37839,145,2
3,17708,37725,145,2
4,17794,37634,145,2


In [49]:
util.qc_annotation_input_path

AttributeError: 'AnnotationProximityTool' object has no attribute 'qc_annotation_input_path'

In [ ]:
mec = DetectorMetricsDK55('DK55',round =1)
mec.calculate_qualification()